In [ ]:
import bs4
import time
from datetime import datetime
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [ ]:
def get_article_html(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content,"html.parser")
    time.sleep(5) 
    return soup

In [ ]:
def get_data_articlebiz_web(soup):
    
    articles = soup.find_all('div',{"class": lambda L: L and L.startswith('mb-4 card')})
    
    # Declaration
    articles_topic = []
    articles_content = []

    for article in articles:
        # Get article's topic
        topic = article.find("a").get_text()
        articles_topic.append(topic)

        # Get article's content
       
        url_of_article = article.find('a')['href']

        soup = get_article_html(url_of_article)

        content = soup.find('div',attrs={'class':'clearfix'}).find_all("p")
        full_content = ""
        if content != None:
            for paragraph in content:
                full_content = full_content + " " + paragraph.get_text().strip()
        articles_content.append(full_content) 
        
    return articles_topic, articles_content

In [ ]:
def insert_data_to_csv(topic, category, content):
    # Create dataframe
    data = pd.DataFrame({
    "topic": topic,
    "category": category,
    "content": content
    })
    data.to_csv(f'articlebiz Scraping-{category}-{datetime.now().strftime("%d-%m-%Y")}.csv', mode='a')

In [ ]:
categories = ["politics", "business", "pets", "news-society", "arts-entertainment", "travel-leisure", "social-issues", "autos-trucks", "health-fitness",
              "shopping", "computers-technology", "finance", "home", "sports-recreations"]

print("Start scraping from articlebiz!")

for category in categories:
    driver = webdriver.Chrome()
    driver.get(f"https://articlebiz.com/category/{category}")
    driver.maximize_window()
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")
    topic, content = get_data_articlebiz_web(soup)
    insert_data_to_csv(topic, category, content)
    next_button = driver.find_element(by=By.LINK_TEXT, value=('Next')).click() 
    for i in range (1, 28):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(10)
        html = driver.page_source
        soup = BeautifulSoup(html,"html.parser")
        topic, content = get_data_articlebiz_web(soup)
        insert_data_to_csv(topic, category, content)
        next_button = driver.find_element(by=By.LINK_TEXT, value=('Next')).click() 
    driver.close()
    
print("Finish scraping from articlebiz.com!")

